In [6]:
%pip install fiona

  Using cached fiona-1.10.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (56 kB)
  Using cached attrs-25.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl.metadata (6.4 kB)
  Using cached cligj-0.7.2-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 135.5 kB/s eta 0:00:0000:0100:05
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import fiona
import geopandas as gpd

# Set SHAPE_RESTORE_SHX config option
os.environ['SHAPE_RESTORE_SHX'] = 'YES'

# Path to the shapefile
shapefile_path = 'sfBay.shp'

# Load the shapefile
gdf = gpd.read_file(shapefile_path)
print(gdf.head())

                                            geometry
0  POLYGON ((-123.02146 38.81852, -123.02158 38.8...
1  POLYGON ((-122.69068 38.31983, -122.6918 38.32...
2  MULTIPOLYGON (((-122.90154 38.65693, -122.9016...
3  MULTIPOLYGON (((-122.57571 38.29709, -122.5754...
4  POLYGON ((-122.69068 38.31983, -122.68842 38.3...


/Users/ulrike_imac_air/projects/maps/maps_env/lib/python3.11/site-packages/pyogrio/raw.py:198: RuntimeWarning: sfBay.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


### Basic Spatial Analysis

In [ ]:
import matplotlib.pyplot as plt

# Plot the geospatial data
gdf.plot()
plt.title('Geospatial Data')
plt.show()

# Calculate the area of each polygon
gdf['area'] = gdf['geometry'].area
print(gdf[['geometry', 'area']].head())

# Calculate the centroid of each polygon
gdf['centroid'] = gdf['geometry'].centroid
print(gdf[['geometry', 'centroid']].head())

# Buffer each polygon by 0.01 degrees
gdf['buffered'] = gdf['geometry'].buffer(0.01)
print(gdf[['geometry', 'buffered']].head())

In [8]:
%pip install geopandas shapely


Note: you may need to restart the kernel to use updated packages.


In [9]:
import geopandas as gpd
from shapely.geometry import Point

# Define locations of interest in San Francisco
data = {
    "name": [
        "Golden Gate Bridge", "Alcatraz Island", "Fisherman's Wharf", "Union Square", "Chinatown",
        "Baker Beach", "Castro District", "Hidden Garden Steps", "Twin Peaks", "Clarion Alley",
        "Mission Dolores Park", "Nob Hill", "Lombard Street", "Transamerica Pyramid", "Salesforce Tower"
    ],
    "latitude": [
        37.8199, 37.8267, 37.8080, 37.7879, 37.7941,
        37.7936, 37.7609, 37.7563, 37.7544, 37.7616,
        37.7596, 37.7930, 37.8021, 37.7952, 37.7897
    ],
    "longitude": [
        -122.4783, -122.4230, -122.4177, -122.4074, -122.4078,
        -122.4835, -122.4350, -122.4661, -122.4477, -122.4216,
        -122.4270, -122.4160, -122.4187, -122.4022, -122.3960
    ]
}

# Create a GeoDataFrame
geometry = [Point(xy) for xy in zip(data["longitude"], data["latitude"])]
gdf = gpd.GeoDataFrame(data, geometry=geometry, crs="EPSG:4326")  # WGS 84 projection

# Save as a shapefile
gdf.to_file("san_francisco_locations.shp", driver="ESRI Shapefile")

print("Shapefile created successfully!")


Shapefile created successfully!


#### Spatial Join Example

In [12]:
import geopandas as gpd

# Load the points shapefile (locations of interest)
points = gpd.read_file("san_francisco_locations.shp")

# Load the polygons shapefile (assuming this represents different city districts or areas)
polygons = gpd.read_file("sfBay.shp")  # Replace with actual polygon file

# Perform a spatial join to find which points fall within polygons
joined = gpd.sjoin(points, polygons, how="inner", predicate="within")

# Display the result
print(joined)


                    name  latitude  longitude                   geometry  \
0     Golden Gate Bridge   37.8199  -122.4783  POINT (-122.4783 37.8199)   
1        Alcatraz Island   37.8267  -122.4230   POINT (-122.423 37.8267)   
2      Fisherman's Wharf   37.8080  -122.4177   POINT (-122.4177 37.808)   
3           Union Square   37.7879  -122.4074  POINT (-122.4074 37.7879)   
4              Chinatown   37.7941  -122.4078  POINT (-122.4078 37.7941)   
5            Baker Beach   37.7936  -122.4835  POINT (-122.4835 37.7936)   
6        Castro District   37.7609  -122.4350   POINT (-122.435 37.7609)   
7    Hidden Garden Steps   37.7563  -122.4661  POINT (-122.4661 37.7563)   
8             Twin Peaks   37.7544  -122.4477  POINT (-122.4477 37.7544)   
9          Clarion Alley   37.7616  -122.4216  POINT (-122.4216 37.7616)   
10  Mission Dolores Park   37.7596  -122.4270   POINT (-122.427 37.7596)   
11              Nob Hill   37.7930  -122.4160    POINT (-122.416 37.793)   
12        Lo

/Users/ulrike_imac_air/projects/maps/maps_env/lib/python3.11/site-packages/pyogrio/raw.py:198: RuntimeWarning: sfBay.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(
/var/folders/1_/6h4xg3ts1cd8htbp9t8wjyh40000gn/T/ipykernel_29685/107034905.py:10: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  joined = gpd.sjoin(points, polygons, how="inner", predicate="within")
